## SQL Grammar
`statement`: select_statement
           | insert_statement
           | update_statement
           | delete_statement
           | create_table_statement
  ##### Определяет типы SQL-операций: SELECT, INSERT, UPDATE, DELETE, CREATE TABLE.

`select_statement`: "SELECT" columns "FROM" table_name ("WHERE" expression)?
  ##### SELECT-запрос с опциональным условием WHERE.

`insert_statement`: "INSERT INTO" table_name "(" columns ")" "VALUES" "(" values ")"
  ##### INSERT-запрос для вставки данных в таблицу.

`update_statement`: "UPDATE" table_name "SET" set_list ("WHERE" expression)?
  ##### UPDATE-запрос для обновления данных в таблице с опциональным условием WHERE.

`delete_statement`: "DELETE FROM" table_name ("WHERE" expression)?
  ##### DELETE-запрос для удаления данных из таблицы с опциональным условием WHERE.

`create_table_statement`: "CREATE TABLE" table_name "(" column_defs ")"
  ##### Запрос на создание таблицы с определением столбцов.

`columns`: "*" | column_list
  ##### Список столбцов для выборки, "*" означает выбор всех столбцов.

`column_list`: NAME ("," NAME)*
  ##### Определение списка столбцов через запятую.

`table_name`: NAME
  ##### Имя таблицы.

`values`: value ("," value)*
  ##### Список значений для вставки.

`set_list`: set_expression ("," set_expression)*
  ##### Список выражений для обновления значений столбцов.

`set_expression`: NAME "=" value
  ##### Выражение присвоения значения столбцу.

`column_defs`: column_def ("," column_def)*
  ##### Определения столбцов таблицы.

`column_def`: NAME data_type
  ##### Определение одного столбца и его типа данных.

`data_type`: "INT" | "VARCHAR" | "BOOLEAN"
  ##### Допустимые типы данных для столбцов.

`value`: NUMBER | STRING | "TRUE" | "FALSE"
  ##### Возможные значения для столбцов.

`expression`: simple_expression
            | simple_expression logical_operator expression
  ##### Логическое выражение, используемое в условиях WHERE.

`simple_expression`: NAME comparison_operator value
  ##### Простое сравнение столбца и значения.

`logical_operator`: "AND" | "OR"
  ##### Логические операторы.

`comparison_operator`: "=" | "!=" | ">" | "<" | ">=" | "<="
  ##### Операторы сравнения.

In [52]:
!pip3 install lark

In [53]:
from lark import Lark, Transformer

In [54]:
from lark import Lark, Transformer, Tree


grammar = """
    start: statement

    statement: select_statement
            | insert_statement
            | update_statement
            | delete_statement
            | create_table_statement

    select_statement: "SELECT" columns "FROM" table_name ("WHERE" expression)?

    insert_statement: "INSERT INTO" table_name "(" columns ")" "VALUES" "(" values ")"

    update_statement: "UPDATE" table_name "SET" set_list ("WHERE" expression)?

    delete_statement: "DELETE FROM" table_name ("WHERE" expression)?

    create_table_statement: "CREATE TABLE" table_name "(" column_defs ")"

    columns: "*" | column_list
    column_list: NAME ("," NAME)*

    table_name: NAME

    values: value ("," value)*

    set_list: set_expression ("," set_expression)*
    set_expression: NAME "=" value

    column_defs: column_def ("," column_def)*
    column_def: NAME data_type

    data_type: "INT" | "VARCHAR" | "BOOLEAN"

    value: NUMBER | STRING | "TRUE" | "FALSE"

    expression: simple_expression
              | simple_expression logical_operator expression
    simple_expression: NAME comparison_operator value

    logical_operator: "AND" | "OR"
    comparison_operator: "=" | "!=" | ">" | "<" | ">=" | "<="

    %import common.CNAME -> NAME
    %import common.NUMBER
    %import common.ESCAPED_STRING -> STRING
    %ignore " "
"""

# Создание парсера
parser = Lark(grammar, start='start', parser='lalr')

class SQLTransformer(Transformer):
    def update_statement(self, items):
        table_name, set_list, *where_clause = items
        update_dict = {
            "table": str(table_name),
            "set": set_list,
        }
        if where_clause:
            update_dict["where"] = where_clause[0]
        return update_dict

    def set_list(self, items):
        return dict(items)

    def set_expression(self, items):

      column_name, value = items
      return (str(column_name), str(value))

    def expression(self, items):
        if len(items) == 1:
            return items[0]
        else:
            # Сложное выражение
            return " ".join(map(str, items))

    def simple_expression(self, items):
        return " ".join(map(str, items))

def parse_sql(sql):
    tree = parser.parse(sql)
    transformed_tree = SQLTransformer().transform(tree)
    return transformed_tree

sql = 'INSERT INTO my_table (id, name) VALUES (1, "Alice")'

sql_parsed = parse_sql(sql)
print(sql_parsed.pretty())

start
  statement
    insert_statement
      table_name	my_table
      columns
        column_list
          id
          name
      values
        value	1
        value	"Alice"



In [55]:
sql = 'INSERT INTO my_table (id, name) VALUES (1, "Alice")'
sql_parsed = parse_sql(sql)
print(sql_parsed)

Tree(Token('RULE', 'start'), [Tree(Token('RULE', 'statement'), [Tree(Token('RULE', 'insert_statement'), [Tree(Token('RULE', 'table_name'), [Token('NAME', 'my_table')]), Tree(Token('RULE', 'columns'), [Tree(Token('RULE', 'column_list'), [Token('NAME', 'id'), Token('NAME', 'name')])]), Tree(Token('RULE', 'values'), [Tree(Token('RULE', 'value'), [Token('NUMBER', '1')]), Tree(Token('RULE', 'value'), [Token('STRING', '"Alice"')])])])])])


In [56]:
sql = 'UPDATE my_table SET name = "Alice", age = 30 WHERE id = 1'
sql_parsed = parse_sql(sql)
print(sql_parsed)

Tree(Token('RULE', 'start'), [Tree(Token('RULE', 'statement'), [{'table': "Tree(Token('RULE', 'table_name'), [Token('NAME', 'my_table')])", 'set': {'name': 'Tree(Token(\'RULE\', \'value\'), [Token(\'STRING\', \'"Alice"\')])', 'age': "Tree(Token('RULE', 'value'), [Token('NUMBER', '30')])"}, 'where': "id Tree(Token('RULE', 'comparison_operator'), []) Tree(Token('RULE', 'value'), [Token('NUMBER', '1')])"}])])
